# Acoustics with Godunov's method

A simple implementation of Godunov's method for constant coefficient acoustics.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *

In [ ]:
from IPython.display import HTML

In [ ]:
try:
    from clawpack.visclaw import animation_tools
except:
    print("Failed to load animation_tools from Clawpack")

### Define the numerical grid

We define a finite volume grid with `num_cells` interior points, indexed `1,2,...,num_cells`.  The cell centers start at `dx/2`.  There is also one "ghost cell" on either side.  This works great in Python since array indexing starts at 0, but in Matlab with 1-based indexing the ghost cell would have index 1.  Also if more than 1 ghost cell is used, the indexing will change.  

In [ ]:
xlower = 0.
xupper = 1.
num_cells = 20
dx = (xupper - xlower)/num_cells

# cell centers, including one ghost cell on either side:
x = arange(xlower-dx/2, xupper+dx, dx)

print('including 2 ghost cells, the grid has %i cells' % len(x))

In [ ]:
K0 = 2
rho0 = 2
c0 = sqrt(K0/rho0)
print('sound speed c0 = %.3f' % c0)
Z0 = sqrt(K0*rho0)
print('impedance Z0 = %.3f' % Z0)

In [ ]:
dt = 1.0*dx/c0
cfl = c0*dt/dx
print('The Courant number is %.3f' % cfl)

Note that Godunov's method should be "exact" if the Courant number is 1 (for constant coefficient acoustics), but unstable for larger values.

## Function to take a single time step:

For simplicity, this function has only two inputs, the arrays of Pn $=P^n$ and Un $=U^n$ containing the pressure and velocity at some time t_n.

It returns the updated arrays Pnp $= P^{n+1}$ and Unp $= U^{n+1}$ at the end of the time step.

This function uses values set above for the material parameters and also for dx and dt.  If you change these values above, you need to re-execute this function definition.

In [ ]:
def Godunov_step(Pn,Un):
    # initialize new cell averages at n+1 to old ones at n:
    # important to make a copy so we can modify without change old values!
    Pnp = Pn.copy()
    Unp = Un.copy()

    # fill ghost cells for periodic BCs:
    # fill at t_n so these values can be used in computing updates
    Pn[0] = Pn[-2]
    Pn[-1] = Pn[1]
    Un[0] = Un[-2]
    Un[-1] = Un[1]
    
    # loop over interfaces where Riemann problem will be solved:
    for i in range(1,len(x)):
        
        # solve Riemann problem between i-1 and i:
        
        # left and right states (using old values at t_n):
        PL = Pn[i-1]; PR = Pn[i]
        UL = Un[i-1]; UR = Un[i]
        
        # coefficients of eigenvectors in W^p = alpha^p * r^p:
        # from (3.31) in FVMHP
        alpha1 = (-(PR-PL) + Z0*(UR-UL)) / (2*Z0)
        alpha2 = ((PR-PL) + Z0*(UR-UL)) / (2*Z0)
        
        # wave speeds (eigenvalues):
        s1 = -c0
        s2 = c0
        
        # left-going wave W1 updates cell i-1:

        Pnp[i-1] = Pnp[i-1] - dt/dx * s1 * alpha1 * (-Z0)
        Unp[i-1] = Unp[i-1] - dt/dx * s1 * alpha1 * 1.
        
        # right-going wave W2 updates cell i:
        Pnp[i] = Pnp[i] - dt/dx * s2 * alpha2 * Z0
        Unp[i] = Unp[i] - dt/dx * s2 * alpha2 * 1.
        
    return Pnp, Unp

### Function to plot a numerical solution at one time:

In [ ]:
def plotQ(Pn,Un,tn):
    subplot(2,1,1)
    plot(x[1:-1], Pn[1:-1], 'bo-')
    xlim(xlower,xupper)
    title('pressure at time %.3f' % tn)
    
    subplot(2,1,2)
    plot(x[1:-1], Un[1:-1], 'bo-')
    xlim(xlower,xupper)
    title('velocity at time %.3f' % tn)
    tight_layout() 

### Initial conditions:

The arrays Pn and Un should be numpy arrays of length `num_cells + 2` with `dtype = float`.  Make sure they aren't integer arrays!

In [ ]:
tn = 0.

#Pn = exp(-150*(x-0.5)**2)
Pn = where(logical_and(x>0.4,x<0.6), 1., 0.)

Un = zeros(x.shape)
#Un = -Pn / Z0  # only left-going wave
#Un = Pn / Z0  # only right-going wave

fig = figure(figsize=(5,5))
plotQ(Pn,Un,tn)

### Time stepping:

Executing the next cell repeatedly will take a time step and plot the new solution

In [ ]:
tn = tn + dt
Pn, Un = Godunov_step(Pn,Un)
fig = figure(figsize=(5,5))
plotQ(Pn,Un,tn)

## Make an animation:

This will only work if you were able to import animation_tools.

In [ ]:
# initial conditions:
tn = 0.
    
#Pn = exp(-150*(x-0.5)**2)
Pn = where(logical_and(x>0.4,x<0.6), 1., 0.)

Un = zeros(x.shape)
#Un = -Pn / Z0  # only left-going wave
#Un = Pn / Z0  # only right-going wave

nsteps = 20  # how many time steps to take
nplot = 1    # how often to make a plot
figs = []    # accumulate figures to animate

for n in range(0,nsteps):
    tn = n*dt
    if mod(n,nplot)==0:
        fig = figure(figsize=(5,5))
        plotQ(Pn,Un,tn)
        figs.append(fig)
        close(fig)
    if n < nsteps-1:
        # take the next step
        Pn, Un = Godunov_step(Pn,Un)


anim = animation_tools.animate_figs(figs, figsize=(5,5))
HTML(anim.to_jshtml())